In [5]:
# imports and keys
import os
import numpy as np
import pandas as pd

from flaml.autogen.agent import AssistantAgent, UserProxyAgent, MathUserProxyAgent
from flaml import oai
from flaml.autogen.code_utils import execute_code

os.environ["WOLFRAM_ALPHA_APPID"] = open("wolfram.txt").read().strip()
config_list = config_list = [{
    'model' : 'gpt-4-0613',
    'api_key': open("key_e.txt").read().strip(),
}]

## Use Wolfram

In [6]:
sys_prompt = """You are an advanced AI with the capability to solve complex math problems.
Wolfram alpha is provided as an external service to help you solve math problems.

When the user gives a math problem, please use the most efficient way to solve the problem.
You are encouraged to use Wolfram alpha whenever it is possible during the solving process. For example, simplications, calculations, equation solving, etc.
However, if the operation requires little computation (very simple calculations, etc), you can also solve it directly.
Reply "TERMINATE" in the end when everything is done.
"""

func_config = {
    "functions": [
        {
            "name": "query_wolfram",
            "description": "Return the API query result from the Wolfram Alpha.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The Wolfram Alpha code to be executed.",
                    }
                },
                "required": ["code"],
            },
        }
    ],
    "function_call": "auto",
}


# function to be passed
def query_wolfram(code):
    mathagent = MathUserProxyAgent()
    return mathagent._execute_one_wolfram_query(code)[0]


chatbot = AssistantAgent("chatbot", sys_prompt, config_list=config_list, **func_config)

user = UserProxyAgent(
    "user",
    human_input_mode="NEVER",
    functions={"query_wolfram": {"function": query_wolfram}},
)

chatbot.receive(
    "Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.",
    user,
)


user (to chatbot):

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

--------------------------------------------------------------------------------
chatbot (to user):

***** Suggested function Call: query_wolfram *****
Arguments: 
{
  "code": "solve (2x+10)(x+3)<(3x+9)(x+8) for x"
}
**************************************************

--------------------------------------------------------------------------------
>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
user (to chatbot):

***** Response from calling function "query_wolfram" *****
Assumption: solve (2 x + 10) (x + 3)<(3 x + 9) (x + 8) for x 
Answer: ans 0: x<-14
ans 1: x>-3

**********************************************************

--------------------------------------------------------------------------------
chatbot (to user):

The solution to the inequality $(2x+10)(x+3)<(3x+9)(x+8)$ is $x<-14$ or $x>-3$.

Expressed in interval not

## Use Python

In [7]:

sys_prompt = """You are an advanced AI with the capability to solve complex math problems.
You can use Python code as an external computing resource. You can write python code to be executed. 
Note:
1. Your previous code will not be saved. So you need to define variables and functions again if you need them.
2. Your have to print the result of your code. Otherwise the result will not be returned.

When the user gives a math problem, please use the most efficient way to solve the problem.
You are encouraged to use Python whenever it is possible during the solving process. For example, simplications, calculations, equation solving, etc.
However, if the operation requires little computation (very simple calculations, etc), you can also solve it directly.

Reply a single "TERMINATE" in the one message when everything is done.
"""
func_config = {
    "functions": [
        {
            "name": "run_python_code",
            "description": "Return a tuple (str, bool): the execution result and if the execution is successful. The printed output or the errors will be returned.",
            "parameters": {
                "type": "object",
                "properties": {
                    "pycode": {
                        "type": "string",
                        "description": "The python code to be executed, with correct indentations.",
                    }
                },
                "required": ["pycode"],
            },
        }
    ],
    "function_call": "auto",
}


chatbot = AssistantAgent("chatbot", sys_prompt, config_list=config_list, **func_config)

user = UserProxyAgent(
    "user",
    human_input_mode="NEVER",
    functions={
        "run_python_code": {
            "class": MathUserProxyAgent(use_docker=False),
            "func_name": "_execute_one_python_code",
        }
    },
)

chatbot.receive(
    "Problem: Find $x^3=125$.",
    user,
)


user (to chatbot):

Problem: Find $x^3=125$.

--------------------------------------------------------------------------------
chatbot (to user):

***** Suggested function Call: run_python_code *****
Arguments: 
{
"pycode": "
import math

# define the equation x^3 = 125
x = math.pow(125, 1/3)
x
"
}
****************************************************

--------------------------------------------------------------------------------
>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
user (to chatbot):

***** Response from calling function "run_python_code" *****
('4.999999999999999\n', True)
************************************************************

--------------------------------------------------------------------------------
chatbot (to user):

The solution to the equation \(x^3 = 125\) is \(x = 5\). 

Is there another problem I could help you with today?

--------------------------------------------------------------------------------
>>>>>>>> NO HUMAN INPUT RECEI